## Langchain Demo: AI Agents on AWS Bedrock

- thinking about an agent AI that integrates with the toolset
- see the reference documentation here: https://python.langchain.com/docs/tutorials/agents/
- in this case, using AWS Bedrock for DeepSeek model access using my named profile in `~/.aws/credentials` file

In [2]:
# Ensure your AWS credentials are configured
import langchain
from langchain.chat_models import init_chat_model
import pandas as pd
import getout_of_text_3 as got3

In [3]:
#model_id='us.deepseek.r1-v1:0' # does not work with tools implementation
model_id='openai.gpt-oss-120b-1:0'
max_tokens=128000 # for openai.gpt-oss-120b-1:0

In [4]:
model = init_chat_model(model_id, 
                        model_provider="bedrock_converse",
                        credentials_profile_name='atn-developer',
                        max_tokens='128000' # maximum for bedrock converse
                        )

In [5]:
query = "Hi! What is the capital of Portugal?"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'The capital of Portugal is **Lisbon**.'

## DIY SCOTUS Corpus Query Tool & Example

- Using `got3` to filter on keywords in a DIY SCOTUS Corpus, via extracting text in PDFs downloaded from **Library of Congress** collection on the [United States Reports](https://www.loc.gov/collections/united-states-reports/).
- Corpus is stored as a **dictionary** of **dataframes**, where each key is a **volume number** and each dataframe contains the cases in that volume.
    - the `case_id` is a concatenated volume and page number (i.e. `329001` is volume 329, page 1). This is also a schema for saving PDF downloads locally. See details at [the U.S. Report page](https://www.supremecourt.gov/opinions/USReports.aspx).

        ```json
        {"329": {DataFrame}, 
        "330": {DataFrame}, ..., 
        "570": {DataFrame}
        }
        ```
        ```text
        case_id	text
        0	570729	OCTOBER \nTERM, 2012 \n729 \nSyllabus \nSEKHAR...
        1	570338	338 \nOCTOBER \nTERM, 2012 \nSyllabus \nUNIVER...
        2	570099	OCTOBER \nTERM, 2012 \n99 \nSyllabus \nALLEYNE...
        ```

In [6]:
# read pdf scotus files
df = pd.read_json("loc_gov.json", lines=True)

df['key'] = df['filename'].apply(lambda x: x.split('usrep')[1][:3])
df['subkey'] = df['filename'].apply(lambda x: x.split('usrep')[1].split('.pdf')[0])

# Create a dictionary to hold the DataFrame contents
df_dict = {}

for _, row in df.iterrows():
    if row['key'] not in df_dict:
        df_dict[row['key']] = {}
    df_dict[row['key']][row['subkey']] = row['content']

# format scotus data for getout_of_text_3, similar to COCA keyword results
db_dict_formatted = {}
for volume, cases in df_dict.items():
    # Create a DataFrame for each volume with case text
    case_data = []
    for case_id, case_text in cases.items():
        case_data.append({'case_id': case_id, 'text': case_text})
    db_dict_formatted[volume] = pd.DataFrame(case_data)


In [7]:
type(db_dict_formatted)

dict

In [8]:
# PRINT LOWEST AND HIGHEST VOLUME NUMBERS
sorted_keys = sorted(db_dict_formatted.keys(), key=lambda x: int(x), reverse=False)
print('VOLUMES', sorted_keys[0], '-', sorted_keys[-1])

VOLUMES 329 - 570


In [9]:
sorted_keys = sorted(db_dict_formatted.keys(), key=lambda x: int(x), reverse=False)
print(sorted_keys)
db_dict_formatted['570'].head()

['329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '430', '431', '432', '433', '434', '435', '436', '437', '438', '439', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '470', '471'

,case_id,text
0,570729,"OCTOBER \nTERM, 2012 \n729 \nSyllabus \nSEKHAR..."
1,570338,"338 \nOCTOBER \nTERM, 2012 \nSyllabus \nUNIVER..."
2,570099,"OCTOBER \nTERM, 2012 \n99 \nSyllabus \nALLEYNE..."
3,570529,"OCTOBER \nTERM, 2012 \n529 \nSyllabus \nSHELBY..."
4,570048,"48 \nOCTOBER \nTERM, 2012 \nSyllabus \nMARACIC..."


## SCOTUS Analysis Tools (LangChain + `getout_of_text_3`)

This section documents two complementary LangChain Tool implementations for exploratory forensic / statutory‐interpretation analysis over a locally prepared SCOTUS corpus.

### 1. Purpose
You can (a) execute an on‑the‑fly keyword search + AI analysis, or (b) feed in *already filtered* JSON results for reproducible, cost‑controlled, deterministic re‑analysis with AI tools. This is for exploratory and demonstrative purposes only; do **NOT** rely on these tools for authoritative legal research or advice.

### 2. Tool Inventory
| Tool Name | Class | Performs Corpus Search? | Input Content Source | Best For | Cost / Token Control | Notes |
|-----------|-------|-------------------------|----------------------|----------|----------------------|-------|
| `scotus_analysis` | `ScotusAnalysisTool` | YES (internal `got3.search_keyword_corpus`) | Raw SCOTUS corpus dict (`db_dict_formatted`) | Quick ad‑hoc exploration, first look | Lower control (dynamic result size) | Returns free‑form model text. Not suited for strict reproducibility. |
| `scotus_filtered_analysis` | `ScotusFilteredAnalysisTool` | NO (analysis only) | Pre‑filtered JSON (already produced elsewhere) | Stable reports, batching, auditing, caching | High control (you decide slice + cap contexts) | Optional structured JSON output (`return_json=True`). |

### 3. Input Schemas (Pydantic)
1. `ScotusAnalysisInput`:
   - `keyword: str` – term/phrase to search.
   - `analysis_focus: str = 'general'` – one of: `general`, `evolution`, `judicial_philosophy`, `custom`.
2. `ScotusFilteredAnalysisInput`:
   - `keyword: str` – label only (no searching performed).
   - `results_json: str | dict` – JSON/dict from a *previous* `got3.search_keyword_corpus` call (after any manual filtering).
   - `analysis_focus: str = 'general'` – same options as above.
   - `max_contexts: int = 60` – hard cap on sampled context snippets injected into the prompt (prevents runaway token bills).
   - `return_json: bool = False` – if `True` the prompt instructs the model to emit **strict JSON** (post‑validated / auto‑repaired if malformed).

### 4. Typical Workflow Patterns
| Scenario | Recommended Flow |
|----------|------------------|
| Rapid hypothesis check | Use `scotus_analysis` with a single keyword, review summary. |
| Iterative refinement / human curation | Run raw search once, manually prune / cluster results, then pass curated JSON to `scotus_filtered_analysis`. |
| Batch reporting (multiple keywords) | Precompute & persist each keyword’s JSON → loop over `scotus_filtered_analysis` with `return_json=True`. |
| Cost‑sensitive environment | Always pre‑filter & throttle with `max_contexts` (e.g. 40–80). |
| Need structured downstream ingestion | Use `return_json=True` and parse validated keys (`reasoning_content`, `summary`, etc.). |

### 5. Prompt Design Overview
Both tools:
- Enforce an “ONLY use supplied contexts” constraint (mitigates hallucination beyond current slice).
- Provide distribution metadata (volumes, counts, occurrence sample) to steer higher‑level synthesis.
- Offer `analysis_focus` to narrow stylistic / topical emphasis.

`scotus_analysis` additionally:
- Embeds a truncated JSON of search hits (first N characters) directly after retrieval.

`scotus_filtered_analysis` adds:
- Multi‑shape normalization for result structures: strings, lists, objects with `context`/`text` fields.
- Safe context sampling + length clipping (`[:240]` chars each) to control token pressure.
- Optional strict JSON response contract.

### 6. Output Shapes
| Mode | Example (abridged) |
|------|--------------------|
| `scotus_analysis` (text) | "Usage Summary... Contextual Patterns..." |
| `scotus_filtered_analysis` (`return_json=False`) | Same narrative section headings as above. |
| `scotus_filtered_analysis` (`return_json=True`) | `{ "keyword": "ordinary meaning", "total_contexts": 217, "occurrences_summary": "217 snippet(s)...", "reasoning_content": ["..."], "summary": "...", "limitations": "..." }` |

### 7. Reliability & Error Handling
- Async safety: attempts event‑loop reuse; falls back to sync if needed.
- JSON robustness: if model returns malformed JSON, a salvage regex pass wraps content into a valid fallback structure.
- Single‑occurrence safeguard: explicitly flags low‑evidence situations (e.g., only 1 context) to prevent over‑extrapolation.

### 8. Performance & Cost Notes
| Driver | Effect | Mitigation |
|--------|--------|------------|
| Large keyword result sets | Longer prompt → higher tokens | Pre‑filter & lower `max_contexts` |
| Very common terms (e.g., "the", functional words) | Noise / inflated contexts | Encourage user to refine / phrase search |
| High `max_contexts` + JSON mode | Larger instructions + response | Tune to 30–80; rarely need >100 contexts |
| Duplicate or near‑duplicate contexts | Redundant token usage | Consider de‑dupe preprocessing before passing JSON |

### 9. When NOT to Use These Tools
- You need full‑text semantic retrieval (vector / embedding) across the corpus → integrate an embedding + retriever pipeline instead.
- You require authoritative legal interpretation beyond provided snippets → domain attorney review required.
- You want cross‑corpus comparative linguistics (e.g., COCA vs SCOTUS) in one call → design a composite prompt / multi‑tool pipeline.

### 10. Minimal Usage Examples
Live search (exploratory):
```python
result_text = scotus_tool._run(keyword="textualism", analysis_focus="judicial_philosophy")
print(result_text[:800])
```
Pre‑filtered (deterministic):
```python
raw_results = got3.search_keyword_corpus(
    keyword="ordinary meaning",
    db_dict=db_dict_formatted,
    case_sensitive=False,
    show_context=True,
    context_words=30,
    output="json"
)
filtered = {k: v for k, v in raw_results.items() if v}  # prune empties
json_str = json.dumps(filtered)
analysis = filtered_scotus_tool._run(
    keyword="ordinary meaning",
    results_json=json_str,
    analysis_focus="general",
    max_contexts=50,
    return_json=True
)
analysis["summary"][:500]
```

### 11. Interpreting Structured JSON (Key Semantics)
| Key | Meaning | Typical Consumer Action |
|-----|---------|-------------------------|
| `keyword` | Echo label for downstream grouping | Index in dataframe / vector store |
| `total_contexts` | Sampled context count (post‑cap) | Assess evidence density |
| `occurrences_summary` | Human‑readable distribution summary | Display directly in UI |
| `reasoning_content` | Stepwise internal analysis (chain‑of‑thought lite) | Optional trust / audit pane |
| `summary` | Main synthesized narrative | Persist / compare across keywords |
| `limitations` | Self‑reported caveats (and JSON salvage note if any) | Flag for review / quality scoring |

### 12. Extension Ideas (Future Work)
- Add semantic clustering (MiniLM embeddings) before sampling to maximize diversity.
- Integrate rate limiting & token accounting dashboards.
- Provide a deterministic hash of the input JSON slice for reproducibility tracking.
- Optional KWIC alignment / colorized keyword highlighting inside samples.

### 13. Quick Decision Guide
> If you can already see and trust the filtered JSON you want analyzed, **use `scotus_filtered_analysis`**. Otherwise, start with `scotus_analysis` to discover whether the keyword is even worth a curated run.

---
**Reminder:** Both tools operate strictly over *your supplied corpus slice*. They intentionally do **NOT** fetch external case law or enrich with outside doctrinal knowledge. This keeps analyses auditable and grounded.


In [26]:
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional, Type, Dict, Any, Union
import json
import asyncio
import re

# ============================================================================
# ORIGINAL live-search tool (kept for reference but description warns users)
# ============================================================================
class ScotusAnalysisInput(BaseModel):
    """Input for SCOTUS case analysis tool (performs a fresh keyword search)."""
    keyword: str = Field(description="The keyword to search for and analyze in SCOTUS cases")
    analysis_focus: Optional[str] = Field(
        default="general", 
        description="Focus of analysis: 'general', 'evolution', 'judicial_philosophy', or 'custom'"
    )

class ScotusAnalysisTool(BaseTool):
    """Tool that SEARCHES the SCOTUS corpus then analyzes.
    NOTE: For pre-filtered JSON results, use ScotusFilteredAnalysisTool instead.
    """
    name: str = "scotus_analysis"
    description: str = (
        "Analyzes SCOTUS cases for a given keyword after performing an internal search. "
        "Do NOT provide pre-filtered results to this tool."
    )
    args_schema: Type[BaseModel] = ScotusAnalysisInput
    model: Any = Field(exclude=True)
    db_dict_formatted: Any = Field(exclude=True)
    
    def __init__(self, model, db_dict_formatted, **kwargs):
        super().__init__(**kwargs)
        self.model = model
        self.db_dict_formatted = db_dict_formatted
    
    def _run(self, keyword: str, analysis_focus: str = "general") -> str:  # noqa: D401
        try:
            loop = asyncio.get_event_loop()
            if loop.is_running():
                import nest_asyncio
                nest_asyncio.apply()
                return asyncio.run(self._arun(keyword, analysis_focus))
        except Exception:
            pass
        return self._sync_run(keyword, analysis_focus)
    
    def _sync_run(self, keyword: str, analysis_focus: str = "general") -> str:
        try:
            print(f"🔍 TOOL(search): Searching SCOTUS database for keyword: '{keyword}'")
            import getout_of_text_3 as got3
            search_results = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=self.db_dict_formatted,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output="json"
            )
            results_dict = {k: v for k, v in sorted(search_results.items(), key=lambda item: int(item[0])) if v}
            if not results_dict:
                return f"No results found for keyword '{keyword}' in the SCOTUS database."
            total_cases = sum(len(cases) for cases in results_dict.values())
            volumes = list(results_dict.keys())
            print(f"📊 TOOL(search): Found {total_cases} cases across {len(volumes)} volumes")
            prompt = self._build_prompt(results_dict, keyword, analysis_focus, volumes, total_cases)
            print(f"🤖 TOOL(search): Sending {len(prompt)} characters to AI model for analysis")
            if len(prompt) > max_tokens:
                print("⚠️ TOOL(search): Prompt exceeds {} characters, which may cause issues with some models.".format(max_tokens))
            response = self.model.invoke([{"role": "user", "content": prompt}])
            print(f"✅ TOOL(search): Analysis complete, returning {len(getattr(response,'content', str(response)))} characters")
            return getattr(response, 'content', str(response))
        except Exception as e:
            error_msg = f"Error analyzing SCOTUS results: {str(e)}"
            print(f"❌ TOOL(search): {error_msg}")
            return error_msg

    async def _arun(self, keyword: str, analysis_focus: str = "general") -> str:
        try:
            print(f"🔍 TOOL(search-async): Searching SCOTUS database for keyword: '{keyword}'")
            import getout_of_text_3 as got3
            search_results = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=self.db_dict_formatted,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output="json"
            )
            results_dict = {k: v for k, v in sorted(search_results.items(), key=lambda item: int(item[0])) if v}
            if not results_dict:
                return f"No results found for keyword '{keyword}' in the SCOTUS database."
            total_cases = sum(len(cases) for cases in results_dict.values())
            volumes = list(results_dict.keys())
            print(f"📊 TOOL(search-async): Found {total_cases} cases across {len(volumes)} volumes")
            prompt = self._build_prompt(results_dict, keyword, analysis_focus, volumes, total_cases)
            print(f"🤖 TOOL(search-async): Sending {len(prompt)} characters to AI model for analysis")
            if len(prompt) > max_tokens:
                print("⚠️ TOOL(search): Prompt exceeds {} characters, which may cause issues with some models.".format(max_tokens))
                prompt = prompt[:max_tokens]
                print(f"🤖 TOOL(search-async): Truncated prompt to {len(prompt)} characters")
            response = await self.model.ainvoke([{"role": "user", "content": prompt}])
            print(f"✅ TOOL(search-async): Analysis complete, returning {len(getattr(response,'content', str(response)))} characters")
            return getattr(response, 'content', str(response))
        except Exception as e:
            error_msg = f"Error analyzing SCOTUS results: {str(e)}"
            print(f"❌ TOOL(search-async): {error_msg}")
            return error_msg

    def _build_prompt(self, results_dict, keyword, analysis_focus, volumes, total_cases) -> str:
        analysis_prompts = {
            "general": f"""
            Instructions:
            You are an AI Agent inside the open-source forensic linguistic tool `getout_of_text_3`.
            Analyze these SCOTUS case search results for the keyword \"{keyword}\" ONLY using the provided data.
            Data summary:
            - Volumes: {', '.join(sorted(volumes, key=int))}
            - Total case occurrences: {total_cases}
            Provide insights on:
            1. Temporal evolution
            2. Contextual variation
            3. Notable intra-dataset patterns (do NOT import outside knowledge)
            4. Interpretive themes relevant to ordinary meaning
            Results (truncated JSON): {json.dumps(results_dict, indent=2)}...
            """,
            "evolution": f"Focus on change over volumes for '{keyword}'.\nData: {json.dumps(results_dict, indent=2)}...",
            "judicial_philosophy": f"Assess patterns of usage that may hint at differing interpretive approaches for '{keyword}'. Use ONLY provided texts. Data: {json.dumps(results_dict, indent=2)}...",
            "custom": f"Comprehensive analysis for '{keyword}'. Use ONLY provided dataset. Data: {json.dumps(results_dict, indent=2)}..."
        }

        return analysis_prompts.get(analysis_focus, analysis_prompts["general"]).strip()

# ============================================================================
# NEW: Pre-filtered JSON analysis tool (does NOT perform search)
# ============================================================================
class ScotusFilteredAnalysisInput(BaseModel):
    """Input for analyzing an already-filtered SCOTUS keyword JSON result set."""
    keyword: str = Field(description="Keyword being analyzed (for labeling only, not for searching).")
    results_json: Union[str, Dict[str, Any]] = Field(
        description="Pre-filtered JSON (or dict) output from got3.search_keyword_corpus AFTER user filtering."
    )
    analysis_focus: Optional[str] = Field(
        default="general", description="'general', 'evolution', 'judicial_philosophy', or 'custom'"
    )
    max_contexts: int = Field(
        default=60, description="Max number of context snippets to sample for prompt (avoid overruns)."
    )
    return_json: bool = Field(
        default=False, description="If True, attempt to return structured JSON with reasoning_content, summary, etc."
    )

class ScotusFilteredAnalysisTool(BaseTool):
    """Analyze ONLY the supplied pre-filtered SCOTUS keyword result JSON.

    IMPORTANT CONSTRAINTS:
    - MUST NOT perform new searches.
    - MUST NOT reference or speculate about cases outside the provided JSON.
    - All observations must derive strictly from the given result set.
    Supports flexible input structures, including:
      {volume: {case_id: context_string}}
      {volume: {case_id: [context_string, ...]}}
      {volume: {case_id: [ {"context": str}, {"context": str} ] }}
    When return_json=True, the model is instructed to emit valid JSON.
    """
    name: str = "scotus_filtered_analysis"
    description: str = (
        "Analyzes pre-filtered SCOTUS keyword search JSON (from got3) without performing any new retrieval."
    )
    args_schema: Type[BaseModel] = ScotusFilteredAnalysisInput
    model: Any = Field(exclude=True)

    def __init__(self, model, **kwargs):
        super().__init__(**kwargs)
        self.model = model

    # --------------- Normalization helper ---------------
    def _normalize_model_content(self, raw: Any) -> str:
        """Convert diverse model.content shapes (list/dict/blocks) into a plain string."""
        if isinstance(raw, str):
            return raw
        if isinstance(raw, list):
            parts = []
            for block in raw:
                if isinstance(block, str):
                    parts.append(block)
                elif isinstance(block, dict):
                    # Common text keys
                    for key in ("text", "content", "value", "message"):
                        val = block.get(key)
                        if isinstance(val, str):
                            parts.append(val)
                            break
                    else:
                        parts.append(str(block))
                else:
                    parts.append(str(block))
            return "\n".join(parts)
        if isinstance(raw, dict):
            # Try typical key
            for key in ("text", "content", "value"):
                if key in raw and isinstance(raw[key], str):
                    return raw[key]
            return json.dumps(raw)
        # Fallback
        return str(raw)

    def _run(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "general",
        max_contexts: int = 60,
        return_json: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        try:
            loop = asyncio.get_event_loop()
            if loop.is_running():
                import nest_asyncio
                nest_asyncio.apply()
                return asyncio.run(
                    self._arun(keyword=keyword, results_json=results_json, analysis_focus=analysis_focus, max_contexts=max_contexts, return_json=return_json)
                )
        except Exception:
            pass
        return self._sync_run(keyword, results_json, analysis_focus, max_contexts, return_json)

    def _sync_run(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "general",
        max_contexts: int = 60,
        return_json: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        try:
            results_dict = self._coerce_results(results_json)
            stats = self._compute_stats(results_dict, keyword)
            prompt = self._build_prompt(keyword, results_dict, stats, analysis_focus, max_contexts, return_json)
            print(f"🤖 TOOL(filtered): Sending {len(prompt)} chars to model (contexts: {stats['total_contexts']}) | return_json={return_json}")
            response = self.model.invoke([{"role": "user", "content": prompt}])
            # Normalize content early
            raw = getattr(response, 'content', None)
            if raw is None and hasattr(response, 'text'):
                try:
                    raw = response.text()
                except Exception:
                    raw = response.text
            content = self._normalize_model_content(raw)
            if return_json:
                return self._postprocess_json(content, results_dict, stats)
            return content
        except Exception as e:
            msg = f"Error (filtered analysis): {e}"
            print(msg)
            return {"error": msg} if return_json else msg

    async def _arun(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "general",
        max_contexts: int = 60,
        return_json: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        try:
            results_dict = self._coerce_results(results_json)
            stats = self._compute_stats(results_dict, keyword)
            prompt = self._build_prompt(keyword, results_dict, stats, analysis_focus, max_contexts, return_json)
            print(f"🤖 TOOL(filtered-async): Sending {len(prompt)} chars to model (contexts: {stats['total_contexts']}) | return_json={return_json}")
            response = await self.model.ainvoke([{"role": "user", "content": prompt}])
            raw = getattr(response, 'content', None)
            if raw is None and hasattr(response, 'text'):
                try:
                    raw = response.text()
                except Exception:
                    raw = response.text
            content = self._normalize_model_content(raw)
            if return_json:
                return self._postprocess_json(content, results_dict, stats)
            return content
        except Exception as e:
            msg = f"Error (filtered analysis async): {e}"
            print(msg)
            return {"error": msg} if return_json else msg

    # ---------------- Internal helpers ----------------
    def _coerce_results(self, results_json: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        if isinstance(results_json, str):
            results_dict = json.loads(results_json)
        else:
            results_dict = results_json
        if not isinstance(results_dict, dict) or not results_dict:
            raise ValueError("results_json must be a non-empty dict or JSON string")
        return results_dict

    def _extract_contexts_from_case(self, occs) -> list:
        """Return list of context strings from flexible case structures."""
        contexts = []
        if isinstance(occs, str):
            contexts.append(occs)
        elif isinstance(occs, dict):
            for k in ("context", "text", "snippet", "kwic"):
                if k in occs and isinstance(occs[k], str):
                    contexts.append(occs[k])
                    break
        elif isinstance(occs, list):
            for o in occs:
                contexts.extend(self._extract_contexts_from_case(o))
        return contexts

    def _compute_stats(self, results_dict: Dict[str, Any], keyword: str) -> Dict[str, Any]:
        volumes = sorted(results_dict.keys(), key=lambda x: int(x) if str(x).isdigit() else str(x))
        case_counts = {}
        total_contexts = 0
        occurrences_per_case = []
        for vol, cases in results_dict.items():
            if not isinstance(cases, dict):
                continue
            case_counts[vol] = len(cases)
            for case_id, occs in cases.items():
                contexts = self._extract_contexts_from_case(occs)
                occ_count = len(contexts)
                total_contexts += occ_count
                occurrences_per_case.append({"volume": vol, "case_id": case_id, "occurrences": occ_count})
        return {
            "volumes": volumes,
            "case_counts": case_counts,
            "total_cases": sum(case_counts.values()),
            "total_contexts": total_contexts,
            "occurrences_per_case": occurrences_per_case,
            "keyword": keyword,
        }

    def _sample_contexts(self, results_dict: Dict[str, Any], max_contexts: int) -> list:
        samples = []
        for vol in sorted(results_dict.keys(), key=lambda x: int(x) if str(x).isdigit() else str(x)):
            cases = results_dict[vol]
            if not isinstance(cases, dict):
                continue
            for case_id, occs in cases.items():
                contexts = self._extract_contexts_from_case(occs)
                for ctx in contexts:
                    cleaned = " ".join(ctx.split())[:240]
                    samples.append(f"[{vol}:{case_id}] {cleaned}")
                    if len(samples) >= max_contexts:
                        return samples
        return samples

    def _build_prompt(
        self,
        keyword: str,
        results_dict: Dict[str, Any],
        stats: Dict[str, Any],
        analysis_focus: str,
        max_contexts: int,
        return_json: bool,
    ) -> str:
        sample_contexts = self._sample_contexts(results_dict, max_contexts)
        focus_instructions = {
            "general": "Provide an overview of usage patterns, semantic ranges, and any interpretive variability.",
            "evolution": "Describe shifts across volumes (treat volume ordering as temporal proxy if applicable).",
            "judicial_philosophy": "Identify internal patterns that might hint at differing interpretive strategies (ONLY within provided data).",
            "custom": "Provide a comprehensive structured analysis (frequency, contextual clusters, potential senses).",
        }
        occ_lines = sorted(
            [f"{o['volume']}:{o['case_id']}={o['occurrences']}" for o in stats["occurrences_per_case"]],
            key=lambda x: x
        )[:80]
        if not sample_contexts:
            sample_contexts = ["(No context strings extracted — verify input JSON structure)"]
        base = f"""
            You are an AI analysis component of `getout_of_text_3`.
            STRICT RULE: Use ONLY the provided JSON contexts. DO NOT introduce external cases, doctrines, or speculative references.
            Keyword: "{keyword}"
            Volumes: {', '.join(stats['volumes'])}
            Total Cases: {stats['total_cases']} | Total Context Snippets: {stats['total_contexts']}
            Occurrences Per Case (sample): {'; '.join(occ_lines)}
            Analysis Focus: {analysis_focus} → {focus_instructions.get(analysis_focus, focus_instructions['general'])}
            Sample Contexts ({len(sample_contexts)}):
            ---
            """ + "\n".join(sample_contexts) + "\n---\n"
        if return_json:
            base += (
                "Return ONLY valid JSON with this exact top-level structure (no extra prose):\n"
                "{\n"
                "  \"keyword\": string,\n"
                "  \"total_contexts\": number,\n"
                "  \"occurrences_summary\": string,\n"
                "  \"reasoning_content\": [string, ...],\n"
                "  \"summary\": string,\n"
                "  \"limitations\": string\n"
                "}\n"
                "Populate reasoning_content with a short step-by-step (3-6 bullets).\n"
                "If only one occurrence, reasoning_content should note insufficient data for variation."
            )
        else:
            base += (
                "Required Output Sections:\n1. Usage Summary\n2. Contextual Patterns / Proto-senses\n3. Frequency & Distribution Observations\n4. Interpretability Notes (ordinary meaning indicators)\n5. Open Questions / Ambiguities\nGround all claims ONLY in the contexts above."
            )
        if stats['total_contexts'] == 1:
            base += "\nNOTE: Only one occurrence detected."
        return base.strip()

    def _postprocess_json(self, content: str, results_dict: Dict[str, Any], stats: Dict[str, Any]) -> Dict[str, Any]:
        # Try direct parse
        parsed = None
        try:
            parsed = json.loads(content)
        except Exception:
            # Attempt to extract JSON substring
            if isinstance(content, str):
                match = re.search(r'{[\s\S]*}', content)
                if match:
                    try:
                        parsed = json.loads(match.group(0))
                    except Exception:
                        parsed = None
        if not isinstance(parsed, dict):
            parsed = {
                "keyword": stats['keyword'],
                "total_contexts": stats['total_contexts'],
                "occurrences_summary": f"{stats['total_contexts']} context snippet(s) across {stats['total_cases']} case(s)",
                "reasoning_content": [
                    "Model did not return valid JSON; wrapped raw text.",
                    "Single occurrence limits distributional inference." if stats['total_contexts']==1 else "Multiple contexts allow limited comparative analysis."
                ],
                "summary": content[:4000] if isinstance(content, str) else str(content)[:4000],
                "limitations": "Auto-wrapped due to invalid JSON from model."
            }
        # Ensure required keys
        for k, default in [
            ("reasoning_content", []),
            ("summary", ""),
            ("occurrences_summary", f"{stats['total_contexts']} snippet(s) across {stats['total_cases']} case(s)"),
            ("limitations", "")
        ]:
            if k not in parsed:
                parsed[k] = default
        return parsed

# Helper: format result so reasoning_content appears LAST under a heading
def format_result_output(result):
    """Return a single string with main answer first and reasoning_content appended at end.
    Format:
    <answer text>

    ## reasoning content
    ```text
    <reasoning>
    ```
    Handles several possible shapes returned by Bedrock / LangChain tool binding.
    """
    try:
        # If it's already a plain string
        if isinstance(result, str):
            return result

        # If it's a dict coming from filtered tool with JSON mode
        if isinstance(result, dict):
            reasoning = []
            # reasoning_content may be list or str
            rc = result.get("reasoning_content")
            if isinstance(rc, list):
                reasoning.append("\n".join(str(x) for x in rc))
            elif isinstance(rc, str):
                reasoning.append(rc)
            elif isinstance(rc, dict) and "text" in rc:
                reasoning.append(rc["text"])
            # The main narrative may live in 'summary' or elsewhere
            main_parts = []
            for key in ("summary", "text", "content"):
                if key in result and isinstance(result[key], str):
                    main_parts.append(result[key])
            main_text = "\n\n".join(p for p in main_parts if p and p.strip())
            reasoning_text = "\n\n".join(r for r in reasoning if r and r.strip())
            if reasoning_text:
                return f"{main_text}\n\n## reasoning content\n```text\n{reasoning_text}\n```" if main_text else f"## reasoning content\n```text\n{reasoning_text}\n```"
            return main_text or str(result)

        # If it's a list of message blocks
        if isinstance(result, list):
            reasoning_segments = []
            answer_segments = []
            for block in result:
                if not isinstance(block, dict):
                    continue
                # Some providers put reasoning in block['reasoning_content']['text']
                rc = block.get("reasoning_content")
                if isinstance(rc, dict) and "text" in rc and rc["text"].strip():
                    reasoning_segments.append(rc["text"].strip())
                elif isinstance(rc, str) and rc.strip():
                    reasoning_segments.append(rc.strip())
                # Capture normal text content
                if "text" in block and isinstance(block["text"], str) and block["text"].strip():
                    answer_segments.append(block["text"].strip())
            # Heuristic: treat the LAST text block as the main answer if multiple present
            if answer_segments:
                main_answer = "\n\n".join(answer_segments)
            else:
                main_answer = ""
            reasoning_text = "\n\n".join(reasoning_segments)
            if reasoning_text:
                return f"{main_answer}\n\n## reasoning content\n```text\n{reasoning_text}\n```" if main_answer else f"## reasoning content\n```text\n{reasoning_text}\n```"
            return main_answer or str(result)
        # Fallback
        return str(result)
    except Exception as e:
        return f"(Formatting error: {e})\n{result}"

## Create the agent by binging the tools and the model together

Updated: We now have two tools bound:
1. `scotus_analysis` performs an internal search.
2. `scotus_filtered_analysis` ONLY analyzes pre-filtered JSON you already produced (no new search).

In [27]:
# Instantiate tools
scotus_tool = ScotusAnalysisTool(model=model, db_dict_formatted=db_dict_formatted)
filtered_scotus_tool = ScotusFilteredAnalysisTool(model=model)

tools = [scotus_tool, filtered_scotus_tool]
model_with_tools = model.bind_tools(tools)

print("Defined: ScotusAnalysisTool (searching) & ScotusFilteredAnalysisTool (pre-filtered with flexible parsing + optional JSON mode, normalized content)")
print(f" ✅ Tools bound to model: {[tool.name for tool in tools]}")
print(f" 👨‍⚖️ SCOTUS database has {len(db_dict_formatted)} volumes")

Defined: ScotusAnalysisTool (searching) & ScotusFilteredAnalysisTool (pre-filtered with flexible parsing + optional JSON mode, normalized content)
 ✅ Tools bound to model: ['scotus_analysis', 'scotus_filtered_analysis']
 👨‍⚖️ SCOTUS database has 242 volumes


## 1st tool: AI will search the corpus for you and analyze results

In [ ]:
#keyword="bovine"
keyword="dictionary"
#keyword="etienne"
#keyword="ordinary meaning"
#keyword="bank"

In [29]:
from time import time

search_keyword = keyword  # reuse previous, or set explicitly like: search_keyword = "dictionary"
analysis_focus = "general"  # options: general | evolution | judicial_philosophy | custom

print(f"[DEMO] Running scotus_analysis (live search) for keyword='{search_keyword}' focus='{analysis_focus}'")
start_time = time()
result_text = scotus_tool._run(keyword=search_keyword, analysis_focus=analysis_focus)
elapsed = time() - start_time

print(f"\n[DEMO] Runtime: {elapsed:.2f}s | Raw result type: {type(result_text).__name__}")

# Unified formatted output (reasoning last)
formatted = format_result_output(result_text)
print("\n=== FORMATTED OUTPUT (reasoning at end) ===\n")
print(formatted[:120])  # safety slice to avoid flooding notebook; adjust if needed

[DEMO] Running scotus_analysis (live search) for keyword='bank' focus='general'
🔍 TOOL(search-async): Searching SCOTUS database for keyword: 'bank'
📊 TOOL(search-async): Found 1964 cases across 242 volumes
🤖 TOOL(search-async): Sending 502989 characters to AI model for analysis
⚠️ TOOL(search): Prompt exceeds 128000 characters, which may cause issues with some models.
🤖 TOOL(search-async): Truncated prompt to 128000 characters
✅ TOOL(search-async): Analysis complete, returning 2 characters

[DEMO] Runtime: 101.60s | Raw result type: list

=== FORMATTED OUTPUT (reasoning at end) ===

**SCOTUS SEARCH‑RESULTS FOR “BANK” (VOL. 329‑390) – FORTH‑LEVEL LINGUISTIC ANALYSIS**  

Below is a compact, data‑only b


### Preview Markdown Report of AI summary

saved as `{keyword}.md`

In [32]:
# Export unified analysis (answer + reasoning content at end) to ONE markdown file
from pathlib import Path

# Ensure we have a result; if not, user can re-run the analysis cell first
try:
    _ = result_text  # noqa: F841
except NameError:
    raise RuntimeError("result_text not defined yet. Run the analysis cell first.")

safe_keyword = "".join(c if c.isalnum() or c in ("-", "_") else "_" for c in keyword.strip()) or "analysis_output"
formatted_output = format_result_output(result_text)
outfile = f"{safe_keyword}.md"
with open(outfile, "w", encoding="utf-8") as f:
    f.write(formatted_output)
print(f"Wrote combined analysis + reasoning to {outfile} (length={len(formatted_output)} chars)")

Wrote combined analysis + reasoning to bank.md (length=17748 chars)


## 2nd tool: Passing pre-filtered JSON results to the filtered analysis tool

- for greater control on the keyword samples

In [33]:
keyword='ordinary meaning'

In [ ]:
loc_results = got3.search_keyword_corpus(
    keyword=keyword,
    db_dict=db_dict_formatted,
    case_sensitive=False,
    show_context=True,
    context_words=30,
    output="json"
)
# Drop keys with empty dicts and sort by keys (as integers)
filtered_sorted_results = {k: v for k, v in sorted(loc_results.items(), key=lambda item: int(item[0])) if v}
#filtered_sorted_results
# print length of 
#print(len(filtered_sorted_results))

if 'filtered_sorted_results' in globals() and filtered_sorted_results:
    filtered_json_str = json.dumps(filtered_sorted_results)
    # Plain text (narrative) mode
    filtered_result_text = filtered_scotus_tool._run(
        keyword=keyword,
        results_json=filtered_json_str,
        analysis_focus='general',
        max_contexts=40,
        return_json=False
    )
    print(filtered_result_text)
else:
    print("No pre-filtered results available to demonstrate filtered analysis tool.")

🤖 TOOL(filtered-async): Sending 12182 chars to model (contexts: 217) | return_json=False


______________________

## Conclusion 🐄 🐍 🧑‍⚖️

### AWS Bedrock + LangChain + `got3` + SCOTUS Corpus Analysis

This demo illustrated how to build custom AI‑powered analysis tools over a specialized legal text corpus using LangChain and AWS Bedrock. By combining dynamic keyword search with focused AI synthesis, we can rapidly explore judicial reasoning patterns in Supreme Court opinions.

### AWS Cost Explorer -- Monitoring Bedrock Costs

Useful `jq` filters for your AWS Cost Explorer output:

> notably costs won't show up immediately, so you may need to wait a day or two after incurring costs to see them reflected in Cost Explorer.

```bash
# Filter for Bedrock services specifically from all services with costs
named_profile=atn-developer
region=us-east-1
start_date=2025-10-01
end_date=2025-10-04

aws ce get-cost-and-usage --time-period Start=$start_date,End=$end_date --granularity DAILY --metrics "BlendedCost" --group-by Type=DIMENSION,Key=SERVICE --region $region --profile=$named_profile | jq '.ResultsByTime[].Groups[] | select(.Keys[0] | test("Bedrock"; "i")) | {service: .Keys[0], cost: .Metrics.BlendedCost.Amount}'
```

#### Cost of AWS Bedrock

- on 10/01/2025 this was: `"cost": "0.03760425"` -- check tomorrow to see the costs from my testing runs today.
- okay various runs cost just about $0.10 `  "cost": "0.10061025"`

In [39]:
# Filter for Bedrock services specifically from all services with costs
!aws ce get-cost-and-usage --time-period Start=2025-09-30,End=2025-10-05 --granularity DAILY --metrics "BlendedCost" --group-by Type=DIMENSION,Key=SERVICE --region us-east-1 --profile=atn-developer | jq '.ResultsByTime[].Groups[] | select(.Keys[0] | test("Bedrock"; "i")) | {service: .Keys[0], cost: .Metrics.BlendedCost.Amount}'

{
  "service": "Amazon Bedrock",
  "cost": "0.10061025"
}
{
  "service": "Amazon Bedrock",
  "cost": "0.04100085"
}
{
  "service": "Amazon Bedrock",
  "cost": "0.03763545"
}
